# A Hybrid Approach Toward Efficient and Accurate Intrusion Detection for In-Vehicle Networks

Implementation of the hybrid approach Intrusion Detection System for Controller Area Network proposed by Zhang et al. (https://ieeexplore.ieee.org/document/9687591) with a rule-based component and a deep neural network.

## Initial configuration

### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive/Tesi/DNN-Hybrid/

### Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import random
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
import logging

print(torch.__version__)

2.0.0+cu118


### Set seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)

### Set device

In [ ]:
# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device used: {}".format(device))

### Suppress warnings

## Data elaboration

### Load Data

#### Google dataset

#### NECST dataset

### Process data

#### Calculate the features

## RULE-BASED
Three rules:
- Valid ID
- Time interval
- valid DLC

*doubts on the valid ID (in the paper there's written that it's in the dbc (and that's ok) or in its absence in the CAN trace of a car, which is very much unlikely that all the IDs available will appear, so it could be potentially dangerous for an IDS to block packets just because they were not seen before in the trace*

In [ ]:
# Valid id rule


In [ ]:
# Time interval

In [ ]:
# valid DLC

## Model

5 hidden layers: [100, 100, 80, 60, 40]</br>
1 output neuron (binary)</br>
ReLu each layer </br>
sigmoid output</br>
binary cross-entropy as loss function</br> 
Adam as optimizer</br>

### Model general settings

In [ ]:
input_dim = 5
output_dim = 1

n = 40 # number of inputs
batch_size = 128
epochs = 50
early_stopping_patience = 5
early_stopping_min_delta = 0
lr = 0.001

### Early stopping 

In [ ]:
# credits to Massaro
# TODO: check if possible the copy otherwise re-implement it

class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [ ]:
"""
early_stopper = EarlyStopper(patience=3, min_delta=10)
for epoch in np.arange(n_epochs):
    train_loss = train_one_epoch(model, train_loader)
    validation_loss = validate_one_epoch(model, validation_loader)
    if early_stopper.early_stop(validation_loss):             
        break
"""

### Build the model

In [ ]:
# import torch.nn as nn

class DNN(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(DNN, self).__init__()
    # 5 input features, binary output
    self.fc1 = nn.Linear(input_dim, 100)
    self.fc2 = nn.Linear(100, 100)
    self.fc3 = nn.Linear(100, 80)
    self.fc4 = nn.Linear(80, 60)
    self.fc5 = nn.Linear(60, 40)
    self.out = nn.Linear(40, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    x = F.sigmoid(self.out(x))
    return x

net = DNN(input_dim, output_dim)
print(net)


DNN(
  (fc1): Linear(in_features=5, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=80, bias=True)
  (fc4): Linear(in_features=80, out_features=60, bias=True)
  (fc5): Linear(in_features=60, out_features=5, bias=True)
  (out): Linear(in_features=40, out_features=1, bias=True)
)


### Train the model

### Plot the model